In [1]:
example_input = """sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew"""

In [2]:
from collections import Counter
from pathlib import Path

def interpret(line):
    NW = line.count("nw")
    NE = line.count("ne")
    SW = line.count("sw")
    SE = line.count("se")
    E = line.count("e") - NE - SE
    W = line.count("w") - NW - SW
    
    # tiles are identified uniquely in X-Y coords as
    #
    #   -3,1      -1,1       1,1    
    #
    #       -2,0       0,0        2,0
    #
    # Rows count by 1's and columns count by 
    # 2's so that column coords stagger by row
    # on odd and even boundaries
    # 
    # This returns the same spot for two different sequences
    # that end up on the same tile yet keeps math in integers
    
    x = 2 * (E - W) + NE + SE - (SW + NW)
    y = NE + NW - (SE + SW)
    return (x, y)
    
def black_tiles(data):
    flipped_tiles = Counter([interpret(line) for line in data.splitlines() ])
    blacks = []
    for key,value in flipped_tiles.items():
        if flipped_tiles[key] %2 == 1:
            blacks.append(key)
    return blacks

In [3]:
len(black_tiles(example_input))

10

In [4]:
my_input = Path('../data/tile_flips.txt').read_text()
blacks = black_tiles(my_input)
len(black_tiles(my_input))

388

# Part 2

In [5]:
def neighbors(tile):
    return [
        (tile[0] + 1, tile[1] + 1),
        (tile[0] + 1, tile[1] - 1),
        (tile[0] - 1, tile[1] + 1),
        (tile[0] - 1, tile[1] - 1),
        (tile[0] + 2, tile[1]),
        (tile[0] - 2, tile[1])
    ]

class TileRules:
    
    def __init__(self, starting):
        self.current = set(starting)
        
    def count_neighbors(self, tile):
        return sum([
            (n in self.current)
            for n in neighbors(tile)
        ])
        
    def play1(self):
        must_consider = list(self.current)
        for tile in self.current:
            must_consider.extend(neighbors(tile))
        must_consider = set(must_consider)
            
        next_round = set()
        for tile in must_consider:
            count = self.count_neighbors(tile)
            
            if (# black tile with one or two black neighbors
                (tile in self.current) and (count in (1,2))
                or
                # white tile with two black neighbors
                (tile not in self.current) and (count == 2)
               ):
                next_round.add(tile)
                
        self.current = next_round
                
    def playn(self, n):
        for _ in range(n):
            self.play1()

In [6]:
g = TileRules(black_tiles(example_input))
g.play1()
len(g.current)

15

In [7]:
g.play1()
len(g.current)

12

In [8]:
start = set(black_tiles(my_input))
g = TileRules(start)
g.playn(100)
len(g.current)

4002